# Test pour prédire le type de cancer (microarray data)
Flückiger Nathan, Vincent Guidoux et Joel Kaufmann

## Préparation des données

### Importation des bibliothèques

In [1]:
import os
import numpy as np
import pylab as plt
import pandas as pd

from Bio import Geo


%matplotlib inline

### Importation des données

In [2]:
def load_geo(myfile):
    handle = open(myfile)
    records = Geo.parse(handle)
    return records

In [3]:
records = load_geo('GDS4296.soft')

# description of each condition (cell type)
id_cond = {}  

cancer_set = set()

# loop over records
for r in records:    
    
    rea = r.entity_attributes
    
    #if rea.has_key('subset_sample_id'): 
    if 'subset_sample_id' in rea:
        
        # find conditions associated with each cell type
        if rea['subset_type']=='disease state':
            sample_list = rea['subset_sample_id'].split(',')
            desc = rea['subset_description']
            cancer_set.add(desc)
            for sample in sample_list:
                id_cond[sample] = desc

    #if rea.has_key('dataset_table_begin'): # it's the gene expression data
    if 'dataset_table_begin' in rea:
        data = r.table_rows

# converts data to an array (more practical to handle)
data = np.array(data) 

**Combien y'a-t-il de cellules cancereuses en tout ?**

In [4]:
len(id_cond)

174

**Comment sont-elles réparties ?**

In [5]:
disease_repartition = {}

for disease in id_cond.values():
    if disease in disease_repartition.keys():
        disease_repartition[disease] = disease_repartition[disease] + 1
    else:
        disease_repartition[disease] = 1
        
disease_repartition

{'Adenocarcinoma': 60,
 'Acute lymphoblastic leukemia': 6,
 'Astrocytoma': 3,
 'Chronic myelogenous leukemia': 3,
 'Carcinoma': 12,
 'Carcinosarcoma': 3,
 'Clear cell carcinoma': 2,
 'Cystoadenocarcinoma': 3,
 'Ductal carcinoma': 3,
 'Glial cell neoplasm': 3,
 'Glioblastoma': 12,
 'Large cell carcinoma': 6,
 'Lymphoma': 3,
 'Malignant amelanotic melanoma': 3,
 'Malignant melanotic melanoma': 11,
 'Melanotic melanoma': 9,
 'Myeloma': 3,
 'Papillary infiltrating ductal carcinoma': 3,
 'Pro myelocytic leukemia': 3,
 'Renal spindle cell carcinoma': 3,
 'Renal cell carcinoma': 9,
 'Small cell bronchioalveolar carcinoma': 3,
 'Squamous cell carcinoma': 2,
 'Hypernephroma': 3,
 'Infiltrating ductal carcinoma': 3}

Aux vues de cette répartition, il nous faut regrouper ces différentes type de cellules, car certains, comme `Squamous cell carcinoma` n'ont que deux cellules, et on ne peut pas se permettre d'en mettre une dans le train-set et une dans le test-set, ce n'est pas assez. Nous allons nous renseigner comment regrouper ces données.